In [121]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re

In [122]:
df_ev=pd.read_csv("eda_ev400.csv")
df_ev.head(10)

,Model,Range_km,Efficiency_Wh_per_km,Weight_kg,Zero_to_Hundred_sec,Battery_kWh,Fastcharge_kW,Cargo_L,Price_EUR,Price_INR,Model_Cleaned,Range_Category
0,Renault Zoe Q90Renault Zoe Q90 Discontinued,245.0,167.0,1555.0,13.5,41.0,33.0,338.0,126.0,11781.0,Renault Zoe Q90,Low
1,Kia e-Soul 39,230.0,170.0,1610.0,9.9,39.2,37.0,315.0,167.0,15614.5,Kia e-Soul 39,Low
2,Opel Combo-e Life XL 50 kWhOpel Combo-e Life X...,200.0,232.0,1813.0,11.7,46.3,78.0,806.0,212.0,19822.0,Opel Combo-e Life XL 50 kWh,Low
3,Renault 4 E-Tech 40kWh 120hpRenault 4 E-Tech 4...,245.0,163.0,1485.0,9.2,40.0,55.0,420.0,116.0,10846.0,Renault 4 E-Tech 40kWh 120hp,Low
4,Toyota Proace City Verso L1 50 kWhToyota Proac...,235.0,213.0,1811.0,11.7,50.0,80.0,775.0,170.0,15895.0,Toyota Proace City Verso L1 50 kWh,Low
5,BMW i3s 120 AhBMW i3s 120 Ah Discontinued,230.0,165.0,1365.0,6.9,37.9,47.0,260.0,183.0,17110.5,BMW i3s 120 Ah,Low
6,Toyota Proace Shuttle L 50 kWhToyota Proace Sh...,180.0,257.0,1989.0,12.1,46.3,78.0,980.0,381.0,35623.5,Toyota Proace Shuttle L 50 kWh,Low
7,Honda e AdvanceHonda e Advance Discontinued,170.0,168.0,1595.0,8.3,28.5,32.0,171.0,242.0,22627.0,Honda e Advance,Low
8,Toyota Proace Verso M 50 kWhToyota Proace Vers...,180.0,257.0,2027.0,12.1,46.3,78.0,603.0,313.0,29265.5,Toyota Proace Verso M 50 kWh,Low
9,Mercedes-Benz eVito Tourer Extra-Long 60 kWhMe...,215.0,279.0,2596.0,12.0,60.0,60.0,1390.0,322.0,30107.0,Mercedes-Benz eVito Tourer Extra-Long 60 kWh,Low


In [123]:
df_ev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 400 non-null    object 
 1   Range_km              400 non-null    float64
 2   Efficiency_Wh_per_km  400 non-null    float64
 3   Weight_kg             400 non-null    float64
 4   Zero_to_Hundred_sec   400 non-null    float64
 5   Battery_kWh           400 non-null    float64
 6   Fastcharge_kW         400 non-null    float64
 7   Cargo_L               400 non-null    float64
 8   Price_EUR             400 non-null    float64
 9   Price_INR             400 non-null    float64
 10  Model_Cleaned         399 non-null    object 
 11  Range_Category        400 non-null    object 
dtypes: float64(9), object(3)
memory usage: 37.6+ KB


In [124]:
# STEP 1 — brand from the start of the string
brand_candidates = [
    "Mercedes-Benz", "DS Automobiles", "Alfa Romeo", "Changan Deepal",
    "Volkswagen", "Lightyear", "Maxus", "VinFast", "Maserati", "Genesis",
    "Peugeot", "Citroën", "Citro", "Renault", "Toyota", "Mercedes",
    "Mitsubishi", "Cadillac", "Leapmotor", "Volvo", "Nissan", "Hyundai",
    "Kia", "BMW", "Audi", "Porsche", "Polestar", "Jaguar", "Lucid",
    "Lotus", "Jeep", "Lancia", "Mazda", "Opel", "Mini", "Smart", "MG",
    "BYD", "GWM ORA", "GWM", "Hongqi", "Zeekr", "XPENG", "NIO", "KGM",
    "Omoda", "CUPRA", "TOGG", "Geely", "Seres", "Alpine", "Ford", "Tesla"
]

# prefer the longest match first (so "DS Automobiles" beats "DS")
brand_candidates = sorted(brand_candidates, key=len, reverse=True)

def _brand_from_model(s):
    s = str(s).strip()
    for b in brand_candidates:
        if s.startswith(b):
            return b
    # fallback: first token
    return s.split()[0] if s else None

df_ev["brand"] = df_ev["Model"].apply(_brand_from_model)


In [133]:
df_ev.head(2)

,brand,model,Range_km,Efficiency_Wh_per_km,Weight_kg,Zero_to_Hundred_sec,Battery_kWh,Fastcharge_kW,Cargo_L,Price_EUR,Price_INR,Range_Category,status
0,Renault,Q90,245.0,167.0,1555.0,13.5,41.0,33.0,338.0,126.0,11781.0,Low,Discontinued
1,Kia,e-Soul-X1,230.0,170.0,1610.0,9.9,39.2,37.0,315.0,167.0,15614.5,Low,NaN


In [126]:
df_ev['status'] = df_ev['Model'].apply(lambda x: 'Discontinued' if 'Discontinued' in str(x) else np.nan)


In [127]:

# df_ev = df_ev.dropna(subset=['Model'])

# drop a full column permanently
df_ev.drop("Model_Cleaned", axis=1, inplace=True)


In [128]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  # Also shows all columns



In [129]:
import re

# 1) Clean function (brand/battery/status/body-type etc. hatao + "firefly firefly..." fix)
def _clean_model(text: str) -> str:
    if not isinstance(text, str):
        return ""
    s = text

    # cut tails like Discontinued/Available/since/from/order
    s = re.sub(r'(Discontinued|Available(?:\s+to\s+order)?|since|from|order).*', '', s, flags=re.IGNORECASE)

    # collapse duplicate concatenation like "Renault Zoe ... Renault Zoe ..."
    s = re.sub(r'([A-Z][A-Za-z0-9\-\s]+)\1', r'\1', s)

    # remove leading brand chunk (1–3 capitalized tokens)
    s = re.sub(r'^(?:[A-Z][A-Za-z\-\.&]+(?:\s+[A-Z][A-Za-z\-\.&]+){0,2})\s+', '', s)

    # remove battery/power specs
    s = re.sub(r'\b\d+(?:\.\d+)?\s?(?:kwh|ah|kw|hp|ps|bhp)\b', '', s, flags=re.IGNORECASE)
    s = re.sub(r'\b\d{2,3}\s?kW\b', '', s, flags=re.IGNORECASE)     # e.g., 160 kW
    s = re.sub(r'\b\d+\s?kW\s+\d+\s?kWh\b', '', s, flags=re.IGNORECASE)

    # remove body/trim/type noise
    s = re.sub(r'\b(Hatchback|Cabrio|Convertible|Coupe|Sedan|Saloon|SUV|MPV|Van|Combi|Kombi|Tourer|Traveller|Caravelle|Transporter'
               r'|Estate|Wagon|XL|L1|L2|L3|L4|Long|Short|M|Extra-Long|Shuttle|Combi|People\s*Carrier'
               r'|AWD|RWD|FWD|4MATIC|xDrive|quattro|Performance|Standard\s*Range|Long\s*Range|Extended\s*Range'
               r'|Twin\s*Motor|Single\s*Motor|Entry|Premium|Plus|Pure|Design|Sport|Luxury)\b',
               '', s, flags=re.IGNORECASE)

    # collapse repeated token like "firefly fireflyfirefly firefly" -> "firefly"
    s = re.sub(r'\b([A-Za-z0-9\-]+)(?:\s*\1\b)+', r'\1', s)

    # remove trailing naked numbers (variant codes we stripped)
    s = re.sub(r'\b\d+\b(?=\s*$)', '', s)

    # normalize spaces/punctuation
    s = re.sub(r'\s{2,}', ' ', s).strip(" .-")
    return s.strip()

# 2) Clean the column WITHOUT dropping rows
df_ev["Model"] = df_ev["Model"].fillna("").apply(_clean_model)

# replace blanks with placeholder to avoid nulls
df_ev["Model"] = df_ev["Model"].replace("", "Unknown Model")

# 3) Make duplicates unique (RULES):
#    - Agar end me L<number> hai -> number ko increment karo (L1, L2, L3...)
#    - Warna "-X1", "-X2", ... laga do
counts = {}
total_counts = df_ev["Model"].value_counts().to_dict()

def _make_unique(name: str) -> str:
    counts[name] = counts.get(name, 0) + 1
    n = counts[name]
    total = total_counts.get(name, 1)

    if total == 1:
        return name  # unique already

    # If ends with L<number> (e.g., "e-Caravelle L1") -> bump number
    mL = re.search(r'\bL(\d+)$', name)
    if mL:
        base = re.sub(r'(\d+)$', '', name)  # drop the trailing digits after L
        start_num = int(mL.group(1))
        return f"{base}{start_num + n - 1}"

    # If ends with <Letter><number> (A1/B2 etc.) -> bump number
    mAlphaNum = re.search(r'\b([A-Z])(\d+)$', name)
    if mAlphaNum:
        letter = mAlphaNum.group(1)
        start_num = int(mAlphaNum.group(2))
        base = re.sub(r'([A-Z]\d+)$', '', name)
        return f"{base}{letter}{start_num + n - 1}"

    # Else add -Xn
    return f"{name}-X{n}"

df_ev["Model"] = df_ev["Model"].apply(_make_unique)


In [130]:
# get all columns
cols = list(df_ev.columns)

# move 'brand' to the first position
if 'brand' in cols:
    cols.insert(0, cols.pop(cols.index('brand')))

# reassign column order
df_ev = df_ev[cols]


In [131]:
# rename only the column 'Model' to 'model'
df_ev.rename(columns={'Model': 'model'}, inplace=True)


In [134]:
df_ev['status'] = df_ev['status'].fillna('Presented')


In [136]:
import random

# possible fake car names list (unique realistic ones)
car_names = [
    "Apex", "Vortex", "Lumina", "Eclipse", "Strider", "Falcon", "Raven", "Spectra", "Orion", "Zenith",
    "Quantum", "Nebula", "Phantom", "Velocity", "Ignis", "Horizon", "Titan", "Pulsar", "Nova", "Drift",
    "Astro", "Vertex", "Mirage", "Fusion", "Cascade", "Inferno", "Lyra", "Vector", "Aquila", "Cresta"
]

# shuffle to ensure uniqueness
random.shuffle(car_names)

# helper function to replace only the 'Unknown Model' part
def replace_unknown(name):
    if "Unknown Model" in name:
        new_name = car_names.pop(0) if car_names else "Phantom"
        return name.replace("Unknown Model", new_name)
    return name

# apply on model column
df_ev['model'] = df_ev['model'].apply(replace_unknown)


In [137]:
df_ev

,brand,model,Range_km,Efficiency_Wh_per_km,Weight_kg,Zero_to_Hundred_sec,Battery_kWh,Fastcharge_kW,Cargo_L,Price_EUR,Price_INR,Range_Category,status
0,Renault,Q90,245.0,167.0,1555.0,13.5,41.0,33.0,338.0,126.0,11781.0,Low,Discontinued
1,Kia,e-Soul-X1,230.0,170.0,1610.0,9.9,39.2,37.0,315.0,167.0,15614.5,Low,Presented
2,Opel,Phantom-X1,200.0,232.0,1813.0,11.7,46.3,78.0,806.0,212.0,19822.0,Low,Discontinued
3,Renault,4 E-Tech-X1,245.0,163.0,1485.0,9.2,40.0,55.0,420.0,116.0,10846.0,Low,Presented
4,Toyota,Verso-X1,235.0,213.0,1811.0,11.7,50.0,80.0,775.0,170.0,15895.0,Low,Presented
5,BMW,i3s-X1,230.0,165.0,1365.0,6.9,37.9,47.0,260.0,183.0,17110.5,Low,Discontinued
6,Toyota,L-X1,180.0,257.0,1989.0,12.1,46.3,78.0,980.0,381.0,35623.5,Low,Discontinued
7,Honda,e Advance,170.0,168.0,1595.0,8.3,28.5,32.0,171.0,242.0,22627.0,Low,Discontinued
8,Toyota,Cascade-X2,180.0,257.0,2027.0,12.1,46.3,78.0,603.0,313.0,29265.5,Low,Presented
9,Mercedes-Benz,eVito-X1,215.0,279.0,2596.0,12.0,60.0,60.0,1390.0,322.0,30107.0,Low,Presented


In [138]:
df_ev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brand                 400 non-null    object 
 1   model                 400 non-null    object 
 2   Range_km              400 non-null    float64
 3   Efficiency_Wh_per_km  400 non-null    float64
 4   Weight_kg             400 non-null    float64
 5   Zero_to_Hundred_sec   400 non-null    float64
 6   Battery_kWh           400 non-null    float64
 7   Fastcharge_kW         400 non-null    float64
 8   Cargo_L               400 non-null    float64
 9   Price_EUR             400 non-null    float64
 10  Price_INR             400 non-null    float64
 11  Range_Category        400 non-null    object 
 12  status                400 non-null    object 
dtypes: float64(9), object(4)
memory usage: 40.8+ KB


In [139]:
# final clean file save karne ke liye
df_ev.to_csv('EDA_cleaned_ev_data.csv', index=False)
print("✅ File saved successfully as 'cleaned_ev_data.csv'")


✅ File saved successfully as 'cleaned_ev_data.csv'
